In [ ]:
!pip install faiss-gpu
!pip install pytorch_metric_learning
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 114.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.7 MB/s eta 0:00:00


In [ ]:
import faiss
from tqdm import tqdm
import numpy as np
import pickle as pkl
from collections import OrderedDict
from typing import Dict
from transformers import AutoModel, AutoTokenizer
from sklearn.decomposition import PCA
import sys
! cp -r /content/drive/MyDrive/sapBERT-DUT-cambridge/src /content/src
from src.model_wrapper import Model_Wrapper
import pandas as pd
import sklearn
from sklearn.model_selection import ParameterGrid
import re

In [ ]:
finetunesets = {
    'nl-wiki_bel-test': 'nl_wiki_bel_traintestsplit',
    'mantra_cleaned' : 'nl_wiki_bel_all'
}

In [ ]:
sys.path.append('/content/drive/MyDrive/sapBERT-DUT-cambridge')
sapBERT_epochs = 0
finetune_epochs = 10
test_dataset = 'nl-wiki_bel-valid'
model_directory_path = f'/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa_sapBERT/{sapBERT_epochs}_epoch/ft_{finetune_epochs}_epoch/{finetunesets[test_dataset]}'

data = pd.read_pickle(f"/content/drive/MyDrive/sapBERT-DUT-cambridge/evaluation/{test_dataset}.pkl")
sentences = data['sentence'].to_list()
mentions = data['mention'].to_list()
if test_dataset == 'nl-wiki_bel-valid':
    trues = data['checked_cuis'].to_list()
else:
    trues = data['cui']
trues = [true.lstrip("['").rstrip("']") for true in trues]

In [ ]:
idx2cui = pkl.load(open('/content/drive/MyDrive/sapBERT-DUT-cambridge/index/id2cui.pkl', 'rb'))
cui2string = pkl.load(open('/content/drive/MyDrive/sapBERT-DUT-cambridge/index/cui_to_string', 'rb'))
relations = pkl.load(open('/content/drive/MyDrive/sapBERT_resources/relations', 'rb'))

In [ ]:
from src.model_wrapper import (
    Model_Wrapper
)

In [ ]:
def get_query_embedding(queries, tokenizer, model):
    bs = 128
    all_reps = []
    for i in tqdm(np.arange(0, len(queries), bs)):
        toks = tokenizer.batch_encode_plus(queries[i:i+bs],
                                        padding="max_length",
                                        max_length=25,
                                        truncation=True,
                                        return_tensors="pt")
        toks_cuda = {}
        for k,v in toks.items():
            toks_cuda[k] = v.cuda()
        output = model(**toks_cuda)

        cls_rep = output[0][:,0,:]

        all_reps.append(cls_rep.cpu().detach().numpy())
    all_reps_emb = np.concatenate(all_reps, axis=0)

    return all_reps_emb

def query_index(queries, tokenizer, model, index, idx2cui, cui2string, pca):
    query_embs = get_query_embedding(queries, tokenizer, model)
    query_embs = np.array(pca.transform(query_embs), dtype=np.float32)

    preds = []

    for query_emb in tqdm(query_embs):
        dist, neighbors = index.search(np.reshape(query_emb, (1,256)), 5)
        dist, neighbors = dist[0], neighbors[0]

        pred = idx2cui[neighbors[0]]
        preds.append(pred)

    return preds

In [ ]:
def getResources(model_directory_path):
    # if basemodel
    if '/medRoBERTa/ft_0_epoch/' in model_directory_path:
        tokenizer = AutoTokenizer.from_pretrained('CLTL/MedRoBERTa.nl')
        model = AutoModel.from_pretrained('CLTL/MedRoBERTa.nl').to('cuda')

        index = faiss.read_index('/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa/ft_0_epoch/index')
        pca = pkl.load(open('/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa/ft_0_epoch/pca', "rb"))

        # index = faiss.read_index('/content/drive/MyDrive/sapBERT-DUT-cambridge/index/index_basemodel_pca')
        # pca = pkl.load(open('/content/drive/MyDrive/sapBERT-DUT-cambridge/basemodel_pca', "rb"))
    else:
        model_wrapper = Model_Wrapper().load_model(
                path=f'{model_directory_path}',
                # path='/content/drive/MyDrive/sapBERT-DUT-cambridge/output_finetune_3epoch',
                max_length=25,
                use_cuda=True,
            )
        tokenizer = model_wrapper.get_dense_tokenizer()
        model = model_wrapper.get_dense_encoder()
        index = faiss.read_index(f'{model_directory_path}/index')
        pca = pkl.load(open(f'{model_directory_path}/pca', "rb"))
        # index = faiss.read_index('/content/drive/MyDrive/sapBERT-DUT-cambridge/finetuned_3epoch/index_pca')
        # pca = pkl.load(open('/content/drive/MyDrive/sapBERT-DUT-cambridge/finetuned_3epoch/pca', "rb"))
    return tokenizer, model, index, pca

In [ ]:
def checkRelations(pred, trues, relations):
    if isinstance(trues, list):
        for true in trues:
            key = f"{true}|{pred}"
            if key in relations:
                return relations[key]
    else:
        key = f"{trues}|{pred}"
        key2 = f"{pred}|{trues}"
        if key in relations:
            return relations[key]
        elif key2 in relations:
            return relations[key2]

    return False

In [ ]:
def checkRelation(preds, trues, relations):
    trues_ = []
    relas = []
    dist_1_relas = []

    for i, pred in enumerate(preds):

        rela = checkRelations(pred, trues[i].split(','), relations)
        current_trues = re.findall('C\d{7}', trues[i])

        if pred in current_trues:
            trues_.append(pred)
            dist_1_relas.append(True)
            relas.append('')
        elif rela:
            trues_.append(current_trues[0])
            dist_1_relas.append(True)
            relas.append(rela)
            # relas.append(True)
        else:
            trues_.append(current_trues[0])
            dist_1_relas.append(False)
            relas.append('')
    return trues_, dist_1_relas, relas

In [ ]:
def predict(model_directory_path, sentences, mentions, idx2cui, cui2string):
    tokenizer, model, index, pca = getResources(model_directory_path)
    preds = query_index(mentions, tokenizer, model, index, idx2cui, cui2string, pca)
    return preds

In [ ]:
def evaluate(preds, trues, relas):
    acc = sklearn.metrics.accuracy_score(checked_trues, preds)
    acc_1dist = sum(relas)/len(relas)

    print(f"Accuracy: {acc}")
    print('1-dist accuracy: ', {acc_1dist})

    return acc, acc_1dist

In [ ]:
def saveResults(sentences, mentions, cui2string, preds, trues, relas, save=False):
    results = []

    for i, true in enumerate(trues):
        try:
            results.append([sentences[i], mentions[i], cui2string[preds[i]], cui2string[true], preds[i], true, relas[i]])
        except:
            print(mentions[i], true, preds[i])
            # continue
    df_results = pd.DataFrame(results, columns=['sentence', 'mention', 'prediction', 'label', 'cui_prediction', 'cui_label', 'relation'])

    if save:
        df_results.to_csv(f'{model_directory_path}/predictions.csv')

        with open(f'{model_directory_path}/preds', 'wb') as f:
            pkl.dump(preds, f)

In [ ]:
results = []

param_grid = {
    '2nd_sapBERT_epochs' : [0, 1, 3, 10],
    '3rd_finetune_epochs' : [0, 1, 3, 10],
    'testsets' : ['nl-wiki_bel-test', 'mantra_cleaned']
}

for c in list(ParameterGrid(param_grid)):
    print(c)
    test_dataset = c['testsets']
    finetune_epochs = c['3rd_finetune_epochs']
    sapBERT_epochs = c['2nd_sapBERT_epochs']

    # if basemodel
    if c['2nd_sapBERT_epochs'] == 0 and c['3rd_finetune_epochs'] == 0:
        model_directory_path = f'/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa/ft_0_epoch/{finetunesets[test_dataset]}'
    # if no 2nd phase pretraining
    elif c['2nd_sapBERT_epochs'] == 0:
        model_directory_path = f'/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa/ft_{finetune_epochs}_epoch/{finetunesets[test_dataset]}'
    # if no finetuning
    elif c['3rd_finetune_epochs'] == 0:
        model_directory_path = f'/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa_sapBERT/{sapBERT_epochs}_epoch/ft_0_epoch'
    else:
        model_directory_path = f'/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa_sapBERT/{sapBERT_epochs}_epoch/ft_{finetune_epochs}_epoch/{finetunesets[test_dataset]}'

    data = pd.read_pickle(f"/content/drive/MyDrive/sapBERT-DUT-cambridge/evaluation/{test_dataset}.pkl")
    sentences = data['sentence'].to_list()
    mentions = data['mention'].to_list()
    if test_dataset == 'nl-wiki_bel-test':
        trues = data['checked_cuis'].to_list()
    else:
        trues = data['cui']
    trues = [true.lstrip("['").rstrip("']") for true in trues]

    preds = predict(model_directory_path, sentences, mentions, idx2cui, cui2string)
    checked_trues, dist_1_relas, relas = checkRelation(preds, trues, relations)
    acc, acc_1dist = evaluate(preds, checked_trues, dist_1_relas)
    saveResults(sentences, mentions, cui2string, preds, checked_trues, relas, True)

    results.append([c['2nd_sapBERT_epochs'], c['3rd_finetune_epochs'], c['testsets'], acc, acc_1dist])


{'2nd_sapBERT_epochs': 0, '3rd_finetune_epochs': 0, 'testsets': 'nl-wiki_bel-test'}


Some weights of RobertaModel were not initialized from the model checkpoint at CLTL/MedRoBERTa.nl and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 480/480 [00:00<00:00, 860.81it/s]


Accuracy: 0.20833333333333334
1-dist accuracy:  {0.39791666666666664}
{'2nd_sapBERT_epochs': 0, '3rd_finetune_epochs': 0, 'testsets': 'mantra_cleaned'}


Some weights of RobertaModel were not initialized from the model checkpoint at CLTL/MedRoBERTa.nl and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 157/157 [00:00<00:00, 901.86it/s]


Accuracy: 0.42038216560509556
1-dist accuracy:  {0.5286624203821656}
{'2nd_sapBERT_epochs': 0, '3rd_finetune_epochs': 1, 'testsets': 'nl-wiki_bel-test'}


...

Accuracy: 0.47770700636942676
1-dist accuracy:  {0.6242038216560509}
{'2nd_sapBERT_epochs': 10, '3rd_finetune_epochs': 10, 'testsets': 'nl-wiki_bel-test'}


100%|██████████| 480/480 [00:00<00:00, 866.24it/s]


Accuracy: 0.29583333333333334
1-dist accuracy:  {0.5104166666666666}
{'2nd_sapBERT_epochs': 10, '3rd_finetune_epochs': 10, 'testsets': 'mantra_cleaned'}


100%|██████████| 157/157 [00:00<00:00, 836.85it/s]


Accuracy: 0.47770700636942676
1-dist accuracy:  {0.6178343949044586}


In [ ]:
df_results = pd.DataFrame(results, columns=['sapBERT_epochs', 'finetune_epochs', 'test_set', 'acc', '1-dist_acc'])
df_results

,sapBERT_epochs,finetune_epochs,test_set,acc,1-dist_acc
0,0,0,nl-wiki_bel-test,0.208333,0.397917
1,0,0,mantra_cleaned,0.420382,0.528662
2,0,1,nl-wiki_bel-test,0.229167,0.429167
3,0,1,mantra_cleaned,0.426752,0.547771
4,0,3,nl-wiki_bel-test,0.225000,0.450000
5,0,3,mantra_cleaned,0.426752,0.547771
6,0,10,nl-wiki_bel-test,0.245833,0.456250
7,0,10,mantra_cleaned,0.407643,0.515924
8,1,0,nl-wiki_bel-test,0.106250,0.179167
9,1,0,mantra_cleaned,0.248408,0.299363


In [ ]:
df_results.to_csv('/content/drive/MyDrive/sapBERT-DUT-cambridge/evaluation/hyperparameter-tuning.csv')

In [ ]:
data

,mention,cui,start_index,end_index,sentence,uniq_cui,checked_cuis
id,,,,,,,
2739,anxiolytisch,['C0040616'],16,28,"Temazepam werkt anxiolytisch, hypnotisch, anti...",C0040616,C0040616
4256,bewegingsziekte,['C0026603'],79,94,De top van het gebouw zwaait tijdens harde win...,C0026603,C0026603
144,darmstelsel,['C0017189'],89,100,"Amber, ook wel ambergris, is een overwegend gr...",C0017189,C0017189
2452,alcoholverslaafden,['C0001956'],104,122,"In januari 2007 heeft project ""BinnenPlaats"" b...",C0001956,C0001956
1027,maxillae,['C0024947'],77,85,In de ronde snuit vormen de tanddragende beend...,C0024947,C0024947
...,...,...,...,...,...,...,...
2445,Vasten,['C0015663'],182,188,"De ""Godsvredebeweging"" die uitging van de Abdi...",C0015663,C0015663
8154,baksoda,['C0074722'],38,45,SodabroodSodabrood is brood dat met baksoda ge...,C0074722,C0074722
4511,nikkel(II),['C0028013'],64,74,Het wordt ook toegepast bij de Nozaki-Hiyama-K...,C0028013,C0028013


from matplotlib import pyplot as plt
_df_0['id'].plot(kind='hist', bins=20, title='id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['start_index'].plot(kind='hist', bins=20, title='start_index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['end_index'].plot(kind='hist', bins=20, title='end_index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3.plot(kind='scatter', x='id', y='start_index', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_4.plot(kind='scatter', x='start_index', y='end_index', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_5['id'].plot(kind='line', figsize=(8, 4), title='id')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_6['start_index'].plot(kind='line', figsize=(8, 4), title='start_index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_7['end_index'].plot(kind='line', figsize=(8, 4), title='end_index')
plt.gca().spines[['top', 'right']].set_visible(False)